<a href="https://colab.research.google.com/github/figgarnold/Eurovision/blob/main/AudioFeatures.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Set Up

Installations:

In [ ]:
# System packages needed for Essentia
!apt-get install -y build-essential libfftw3-dev libeigen3-dev libsamplerate0-dev

# Clone Essentia and build it
!git clone https://github.com/MTG/essentia.git
%cd essentia
!./waf configure --with-python
!./waf build
!./waf install

# Add Essentia to Python path
import sys
sys.path.append("/usr/local/lib/python3.8/dist-packages")  # adjust if needed

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
build-essential is already the newest version (12.9ubuntu3).
libfftw3-dev is already the newest version (3.3.8-2ubuntu8).
libsamplerate0-dev is already the newest version (0.2.2-1build1).
libeigen3-dev is already the newest version (3.4.0-2ubuntu2).
0 upgraded, 0 newly installed, 0 to remove and 41 not upgraded.
fatal: destination path 'essentia' already exists and is not an empty directory.
/content/essentia
/content/essentia/wscript:225: SyntaxWarning: invalid escape sequence '\w'
  # libs_paths = [';packaging\win32_3rdparty\\' + lib + '\lib\pkgconfig' for lib in libs_3rdparty]
Setting top to                           : /content/essentia 
Setting out to                           : /content/essentia/build 
→ configuring the project in /content/essentia
→ Building in release mode
Checking for 'g++' (C++ compiler)        : /usr/bin/g++ 
Checking for 'gcc' (C compiler)          : /usr/bin/gcc

In [1]:
!pip install Essentia

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 3.6 MB/s eta 0:00:00


In [2]:
!pip install yt_dlp
!apt-get install -y nodejs



     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 180.0/180.0 kB 4.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.3/3.3 MB 50.7 MB/s eta 0:00:00
Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
The following additional packages will be installed:
  javascript-common libc-ares2 libjs-highlight.js libnode72 nodejs-doc
Suggested packages:
  apache2 | lighttpd | httpd npm
The following NEW packages will be installed:
  javascript-common libc-ares2 libjs-highlight.js libnode72 nodejs nodejs-doc
0 upgraded, 6 newly installed, 0 to remove and 41 not upgraded.
Need to get 13.7 MB of archives.
After this operation, 54.0 MB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu jammy/main amd64 javascript-common all 11+nmu1 [5,936 B]
Get:2 http://archive.ubuntu.com/ubuntu jammy/universe amd64 libjs-highlight.js all 9.18.5+dfsg1-1 [367 kB]
Get:3 http://archive.ubuntu.com/ubuntu jammy-updates/main amd64 libc-ares2 

Imports:

In [3]:
import yt_dlp
import pandas as pd
import essentia.standard as es
import os
import glob

In [4]:
print(os.getcwd())

/content


In [14]:
df = pd.read_csv("/content/tokenisedlyrics.csv")
df = df[df["top3"] == True]

# Cookies

In [17]:
from google.colab import files
uploaded = files.upload()  # choose cookies.txt

Saving cookies.txt to cookies.txt


# Helper Functions

Download Helper:

In [12]:
def download_audio(youtube_url, base_name, cookies_file="cookies.txt", cookie_from_browser=None):
    """
    Download audio from YouTube as WAV using yt-dlp.
    Always returns the correct .wav filename.
    """
    ydl_opts = {
        'format': 'bestaudio/best',
        'outtmpl': f'/content/audio/%{base_name}.%(ext)s',
        'postprocessors': [{
            'key': 'FFmpegExtractAudio',
            'preferredcodec': 'wav',
            'preferredquality': '192',
        }],
        'extractor_args': {'youtube': {'player_client': ['default']}},
        'quiet': True,
        'ignoreerrors': True
    }
    if cookies_file:
        ydl_opts['cookiefile'] = cookies_file
    if cookie_from_browser:
        ydl_opts['cookiefrombrowser'] = cookie_from_browser

    with yt_dlp.YoutubeDL(ydl_opts) as ydl:
        info = ydl.extract_info(youtube_url, download=True)
        final_path = ydl.prepare_filename(info)
        # force extension to .wav
        final_path = os.path.splitext(final_path)[0] + ".wav"
        return final_path

Feature Helper:

In [13]:
def extract_essentia_features(audio_file):
    audio = es.MonoLoader(filename=audio_file)()

    danceability = es.Danceability()(audio)
    bpm, _, _, _, rhythm_strength = es.RhythmExtractor2013(method="multifeature")(audio)

    tonal_features = es.TonalExtractor()(audio)
    key = tonal_features[0]
    scale = tonal_features[1]

    return {
        "danceability": danceability,
        "bpm": bpm,
        "rhythm_strength": rhythm_strength,
        "key": key,
        "scale": scale
    }


# Pipeline

Batch Run:

In [21]:
results = []
batch_size = 10
cookies_path = "cookies.txt"  # optional, if uploaded

# resume at a specific index if needed
start_index = 200

for start in range(start_index, len(df), batch_size):
    batch = df.iloc[start:start+batch_size]
    print(f"Processing batch {start}–{start+len(batch)-1}")

    for idx, row in batch.iterrows():
        url = row["youtube_url"]
        title = str(row.get("title", f"track_{idx}"))

        try:
            audio_path = download_audio(url, base_name=title, cookies_file=cookies_path)

            if not os.path.exists(audio_path):
                print(f"✗ No audio file created for row {idx} (title='{title}')")
                continue

            feats = extract_essentia_features(audio_path)
            feats["title"] = title
            feats["youtube_url"] = url
            results.append(feats)

            os.remove(audio_path)  # free disk space
            print(f"✓ {title} processed (index {idx})")

        except Exception as e:
            print(f"Error with {title} (index {idx}): {e}")
            continue

    # Save after each batch
    pd.DataFrame(results).to_csv("features_partial.csv", mode="a", header=False, index=False)
    results = []  # clear memory before next batch
    print(f"✔ Batch {start}–{start+len(batch)-1} saved")

Processing batch 140–149


✓ track_809 processed (index 809)


✓ track_815 processed (index 815)


✓ track_826 processed (index 826)


✓ track_831 processed (index 831)


...

✓ track_1613 processed (index 1613)


✓ track_1622 processed (index 1622)


✓ track_1626 processed (index 1626)
Error with track_1651 (index 1651): expected string or bytes-like object, got 'float'


✓ track_1653 processed (index 1653)


KeyboardInterrupt: 

# Consolidate

Merge features into original data frame and output new csv:

In [ ]:
all_files = glob.glob("features_partial*.csv")
dfs = [pd.read_csv(f) for f in all_files]
final_df = pd.concat(dfs, ignore_index=True)

final_df.to_csv("features_all.csv", index=False)
print(final_df.head())

In [ ]:
lyrics_df = pd.read_csv("translatedlyrics.csv")
features_df = pd.read_csv("features_all.csv")

print(lyrics_df.columns)
print(features_df.columns)


In [ ]:
merged_df = pd.merge(
    lyrics_df,
    features_df,
    on=["title", "youtube_url"],   # join on both to be safe
    how="left"                     # keep all lyrics, even if audio missing
)

In [ ]:
merged_df.to_csv("lyrics_audio_merged.csv", index=False)
print(merged_df.head())